OBLIVIOUS TRANSFER

Suppose that Alice wants to send two messages to Bob. Also, She wants to give the option to Bob to choose one of Her messages but She also wants to forbid the simultaneous reading of the two messages by Bob. That is, Bob can only read one of the messages depending on Bob's choice. Now, Bob doesn't want to make his choice visible for Alice. That is, only Bob can know what was his choice. To solve this problem we have to make the assumption that Alice and Bob will play by the rules no matter what. That is, Alice and Bob won't try to find the secret information of the other person.



Importing Libraries

In [4]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from random import randint
import hashlib

Funtions to convert strings to and from binary

In [5]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return ''.join([str(x) for x in result])

def frombits(bits):
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

Sender side (Alice)

In [21]:
# Converting the string into binary
messageToSend1 = tobits(input("Enter  message 1 to encrypt:"))
messageToSend1
# Converting the string into binary
messageToSend2 = tobits(input("Enter message 2 to encrypt:"))
messageToSend2

messageMerge = messageToSend1+messageToSend2
messageMerge 


Enter  message 1 to encrypt: Ananya
Enter message 2 to encrypt: Thakre


'010000010110111001100001011011100111100101100001010101000110100001100001011010110111001001100101'

Alice quantum circuit

In [22]:
#Alice cell, Bob can't see what's going in on here
Alice_bases = [randint(0,1) for x in range(len(messageMerge))]
qubits = list()
for i in range(len(Alice_bases)):
    mycircuit = QuantumCircuit(1,1)
    if(Alice_bases[i] == 0):
        if(messageMerge[i] == "1"):
            mycircuit.x(0)
    else:
        if(messageMerge[i] == "0"):
            mycircuit.h(0)
        else:
            mycircuit.x(0)
            mycircuit.h(0)
    qubits.append(mycircuit)

Receiver's Side (Bob)

In [23]:
#Bob cell, Alice can't see what's going in on here
Bob_bases = [randint(0,1) for x in range(len(messageMerge))]
backend = Aer.get_backend('qasm_simulator')
measurements = list()
choice = 0
for i in range(len(Bob_bases)):
    qubit = qubits[i]
    if(Bob_bases[i] == 0):
        qubit.measure(0,0)
    else:
        qubit.h(0)
        qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))

In [24]:
#Bob cell, Alice can't see what's going in on here
I0 = list()
I1 = list()
if(choice == 0):    
    for i in range(len(Alice_bases)):
        if(Alice_bases[i] == Bob_bases[i]):
            I0.append(i)
        else:
            I1.append(i)
else:
    for i in range(len(Alice_bases)):
        if(Alice_bases[i] == Bob_bases[i]):
            I1.append(i)
        else:
            I0.append(i)

In [26]:
#Alice cell, Bob can't see what's going in on here
x0 = list()
for x in I0:
    x0.append(messageMerge[x])
x1 = list()
for x in I1:
    x1.append(messageMerge[x])
fx0 = ''.join(format(ord(i), 'b') for i in hashlib.sha224(''.join(x0).encode('utf-8')).hexdigest())
fx1 = ''.join(format(ord(i), 'b') for i in hashlib.sha224(''.join(x1).encode('utf-8')).hexdigest())
s0 = ''
s1 = ''
for bit in range(len(messageToSend1)):
    s0 += str(int(fx0[bit]) ^ int(messageToSend1[bit]))
for bit in range(len(messageToSend2)):
    s1 += str(int(fx1[bit]) ^ int(messageToSend2[bit]))

In [27]:
xB0 = list()
if(choice == 0):
    for x in I0:
        xB0.append(measurements[x])
else:
    for x in I1:
        xB0.append(measurements[x])
fxB0 = ''.join(format(ord(i), 'b') for i in hashlib.sha224(''.join([str(x) for x in xB0]).encode('utf-8')).hexdigest())
mB0 = bytearray()
if(choice == 0):
    for bit in range(len(s0)):
        mB0.append(int(fxB0[bit]) ^ int(s0[bit]))
    print("Alice message was: " + frombits(mB0))
else:
    for bit in range(len(s1)):
        mB0.append(int(fxB0[bit]) ^ int(s1[bit]))
    print("Alice message was: " + frombits(mB0))

Alice message was: Ananya
